## Initialize Fiddler Client


In [ ]:
import fiddler as fdl
client = fdl.FiddlerApi()

## Load baseline

In [ ]:
import pandas as pd
baseline_df = pd.read_csv('/app/fiddler_samples/samples/datasets/housing/housing-prices.csv')
baseline_schema = fdl.DatasetInfo.from_dataframe(baseline_df, max_inferred_cardinality=1000)


In [ ]:
baseline_df

## Setup monitoring

In [ ]:
project_id = 'tutorial'
name = 'housing_prices'
target='SalePrice'
input_df = baseline_df.drop(columns=['Id', target])
features = list(input_df.columns)

In [ ]:
## setup/cleanup project
if project_id in client.list_projects():
    client.delete_model(project_id, name)
    client.delete_dataset(name)
else:
    client.create_project(project_id)


client.create_surrogate_model(
    project_id,
    name,
    baseline_df,
    baseline_schema,
    target,
    features
)

## Send monitoring events

In [ ]:
import datetime
import random
from IPython.display import clear_output
import time

chunked_df = [input_df[i:i+1] for i in range(0, input_df.shape[0], 1)]
count = 0
for row_df in chunked_df:
    client.run_model(project_id, name, row_df, log_events=True)
    count += 1
    clear_output(wait = True)
    print(f'Sending {count} / {len(chunked_df)}')
    time.sleep(2)